In [5]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import feather
import gc
from datetime import datetime
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
import lightgbm as lgb
from sklearn import tree
from sklearn.metrics import roc_auc_score
from numba import jit
import joblib

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [3]:
%%time
data = pd.read_feather("finalData.ft")

Wall time: 6.54 s


In [4]:
%%time

features = data.columns.tolist()
features.remove('HasDetections')

labels = np.asarray(data['HasDetections'])
data.drop(['HasDetections'],inplace=True,axis=1)


data = np.asarray(data)

Wall time: 21.2 s


In [6]:
@jit(parallel=True)
def train(model,d,l):
    model.fit(d,l)

In [ ]:
%%time
from sklearn.metrics import accuracy_score , confusion_matrix
from sklearn.metrics import accuracy_score

folds = KFold(n_splits=3, shuffle=True)
scores=[]
confusionMatrices=[]
bestscore=0
for (train_index, test_index) in tqdm(folds.split(data,labels)):
    xtrain , ytrain = data[train_index] , labels[train_index]
    xtest , ytest = data[test_index] , labels[test_index]
    
    classifier = tree.DecisionTreeClassifier()
    train(classifier,xtrain,ytrain)
    pred = classifier.predict(xtest)
    score = roc_auc_score(ytest, pred)
    if(score>bestscore):
        bestscore=score
        joblib.dump(classifier, 'decissionTree.pkl')
    confusionMatrices.append(confusion_matrix(ytest,pred))
    scores.append(score)
    
print (scores)




0it [00:00, ?it/s]

In [12]:
print (score)

0.5750778486289954
